
# NetworkX Basics (Modern API, 3.x-safe)

This notebook demonstrates common NetworkX tasks using current, non-deprecated APIs:
- Build a graph with node/edge attributes
- Inspect structure (sizes, density, connectivity)
- Shortest paths
- Centrality
- Community detection (greedy modularity)
- Visualization with Matplotlib
- Save & load (GraphML)
- Small directed example (PageRank)

**Note:** `nx.info(G)` was deprecated/removed in 3.0. We use `print(G)` and direct property calls like `G.number_of_nodes()` instead.


## Setup

In [ ]:

# If NetworkX/Matplotlib are missing, uncomment:
# !pip install networkx matplotlib

import networkx as nx
import matplotlib.pyplot as plt

print("NetworkX version:", nx.__version__)


## Create a small example graph

In [ ]:

G = nx.Graph(name="Friendship Graph")

# Nodes with attributes
G.add_node("Alice",  age=29, city="Madrid")
G.add_node("Bob",    age=31, city="Madrid")
G.add_node("Chloe",  age=28, city="Valencia")
G.add_node("Diego",  age=35, city="Sevilla")
G.add_node("Emma",   age=33, city="Madrid")
G.add_node("Farid",  age=29, city="Bilbao")

# Edges with attributes
G.add_edge("Alice", "Bob",   since=2020)
G.add_edge("Alice", "Chloe", since=2022)
G.add_edge("Bob",   "Chloe", since=2019)
G.add_edge("Bob",   "Emma",  since=2018)
G.add_edge("Chloe", "Diego", since=2021)
G.add_edge("Emma",  "Farid", since=2020)

# A tiny separate component
G.add_node("Gina", age=27, city="Zaragoza")
G.add_node("Hugo", age=30, city="Zaragoza")
G.add_edge("Gina", "Hugo", since=2024)

# Modern way to show a concise summary (replacement for deprecated nx.info(G)):
print(G)  # e.g., "Graph with 8 nodes and 7 edges"
print("Nodes (with data):", G.nodes(data=True))
print("Edges (with data):", G.edges(data=True))


## Basic stats

In [ ]:

n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()
density = nx.density(G)

print(f"Nodes: {n_nodes}, Edges: {n_edges}, Density: {density:.3f}")
print("Is connected?:", nx.is_connected(G))
print("Connected components:", [sorted(c) for c in nx.connected_components(G)])

# Degrees
degrees = dict(G.degree())
print("Degree by node:", degrees)


## Shortest paths

In [ ]:

source, target = "Alice", "Diego"
print(f"Shortest path {source}->{target}:", nx.shortest_path(G, source, target))

# All-pairs (ok for small graphs)
lengths = dict(nx.all_pairs_shortest_path_length(G))
print("Shortest path lengths from Alice:", lengths["Alice"])


## Centrality

In [ ]:

deg_cent = nx.degree_centrality(G)
bet_cent = nx.betweenness_centrality(G, normalized=True)
clo_cent = nx.closeness_centrality(G)

def top_k(d, k=3):
    return sorted(d.items(), key=lambda x: x[1], reverse=True)[:k]

print("Top degree centrality:", top_k(deg_cent))
print("Top betweenness centrality:", top_k(bet_cent))
print("Top closeness centrality:", top_k(clo_cent))


## Community detection (greedy modularity)

In [ ]:

# Import path consistent with docs; top-level import also works.
from networkx.algorithms.community import greedy_modularity_communities

communities = list(greedy_modularity_communities(G))
for i, comm in enumerate(communities, start=1):
    print(f"Community {i}: {sorted(comm)}")

# Map each node -> community index for plotting
node2comm = {}
for i, comm in enumerate(communities):
    for node in comm:
        node2comm[node] = i


## Visualization

In [ ]:

pos = nx.spring_layout(G, seed=42)

# Node sizes by degree centrality
sizes = [3000 * deg_cent[n] for n in G.nodes()]

plt.figure(figsize=(8, 6))
nx.draw_networkx_nodes(G, pos, node_size=sizes, node_color=[node2comm[n] for n in G.nodes()])
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos, font_size=10)

edge_labels = {(u, v): d.get("since", "") for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

plt.title(G.graph.get("name", "Graph"))
plt.axis("off")
plt.show()


## Save & load (GraphML)

In [ ]:

graphml_path = "friendship_graph.graphml"
nx.write_graphml(G, graphml_path)
print("Saved:", graphml_path)

G2 = nx.read_graphml(graphml_path)
print("Loaded graph has", G2.number_of_nodes(), "nodes and", G2.number_of_edges(), "edges.")


## Modify the graph

In [ ]:

G.add_node("Iris", age=26, city="Madrid")
G.add_edge("Iris", "Alice", since=2025)
G.remove_node("Hugo")

print("After modifications:")
print("Nodes:", sorted(G.nodes()))
print("Edges:", sorted(G.edges()))


## Directed example + PageRank

In [ ]:

DG = nx.DiGraph(name="Follower Graph")
DG.add_edges_from([
    ("Alice", "Bob"),
    ("Bob", "Chloe"),
    ("Chloe", "Alice"),
    ("Emma", "Bob"),
    ("Farid", "Bob"),
    ("Diego", "Chloe"),
])

pr = nx.pagerank(DG, alpha=0.85)
print("PageRank:", dict(sorted(pr.items(), key=lambda x: x[1], reverse=True)))
